# Passenger Screening Algorithm Challenge
Camilo Ruiz, Ana-Maria Istrate.  
9 November 2017  
  
**Table of Contents:**
1. Introduction
2. Data Set Exploration
3. Current Work
4. Proposed Work

# 1. Introduction

# 2. Data Set Exploration

# 3. Current Work

## 3.1 A Basic Logistic Regression Model

## 3.2 A ConvNet Model

## 3.3 A Modified ConvNet Model

# 4. Proposed Work